In [6]:
import json
from enum import Enum
import math
import subprocess
import os

def run_command_no_output(command):
    try:
        subprocess.check_call(command)
        return True
    except subprocess.CalledProcessError:
        print(f"Failed to run: {' '.join(command)}")
        return False

class Metric(Enum):
    server_runtime_ms = 0
    max_payload_size_B = 1
    query_size_B = 2
    response_size_B = 3

rlwe_dict = {
    Metric.max_payload_size_B : (lambda data: data['Item size (B)']),
    Metric.server_runtime_ms : (lambda data: data['server time (ms)']),
    Metric.query_size_B : (lambda data: data['request size (B)']/2),
    Metric.response_size_B : (lambda data: data['response size (B)']),

}

spiral_dict ={
    Metric.max_payload_size_B : (lambda data: data['Item size (B)']),
    Metric.server_runtime_ms : (lambda data: data['total_us']/1000),
    Metric.query_size_B : (lambda data: (data['param_sz'] + data['query_sz'])),
    Metric.response_size_B : (lambda data: data['resp_sz']),
}

functions = {
    'RLWE_All_Keys' : rlwe_dict,
    'RLWE_Whispir_3_Keys' : rlwe_dict,
    'RLWE_Whispir_2_Keys' : rlwe_dict,
    'sealpir' : { 
        Metric.max_payload_size_B : (lambda data: data['max_element_size']),
        Metric.server_runtime_ms : (lambda data: data['PIRServer_reply_generation_time']),
        Metric.query_size_B : (lambda data: data['size_gal_keys'] + data['query_size']),
        Metric.response_size_B : (lambda data: data['reply_size']),
    }, 
    'fastpir' : {
        Metric.max_payload_size_B : (lambda data: data['Item size (B)']),
        Metric.server_runtime_ms : (lambda data: data['Response generation time (us)']/1000),
        Metric.query_size_B : (lambda data: data['Query size (B)']),
        Metric.response_size_B : (lambda data: data['Response size (B)']),
    },
    'onionpir' : {
        Metric.max_payload_size_B : (lambda data: data['Item size (B)']),
        Metric.server_runtime_ms : (lambda data: data['Reply time (ms)']),
        Metric.query_size_B : (lambda data: (data['Gal Keys (KB)'] + data['Encrypted Secret Key (KB)'] + data['Query size (KB)']) * 1024),
        Metric.response_size_B : (lambda data: data['Reply size (KB)'] * 1024),
    },
    'spiral' : spiral_dict,
    'spiral-pack' : spiral_dict,
    'spiral-stream' : spiral_dict,
    'spiral-stream-pack' : spiral_dict,
}

run_command_no_output(["mkdir", "-p", 'results'])

TARGET_PAYLOAD_SIZE_B = 256*1024
for name in functions.keys():
    with open(f"results/{name}.csv", "w") as f:
        f.write("num_rows, target_payload_size_B, runtime_s, communication_KB\n")
    for log_num_rows in [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]:
        num_rows = 2**log_num_rows
        directory_to_logs = f"logs/logs-{num_rows}-0/{name}"
        # path_to_file = f"logs/logs-{num_rows}-0/{name}.json"
        all_files = os.listdir(directory_to_logs)
        # Filter to include only .json files
        json_files = [file for file in all_files if file.endswith('.json')]

        # Iterate over each json file
        for json_file in json_files:
        # if os.path.exists(path_to_file):
            path_to_file = os.path.join(directory_to_logs, json_file)
            with open(path_to_file) as f:
                data = json.load(f)
                funcs = functions[name]
                try:
                    SCALAR = math.ceil(TARGET_PAYLOAD_SIZE_B / funcs[Metric.max_payload_size_B](data))
                except Exception as e:
                    SCALAR = None
                # runtime
                try:
                    runtime_s = (funcs[Metric.server_runtime_ms](data) * SCALAR) / 1000
                except Exception as e:
                    runtime_s = None
                
                try:
                    communication_KB = (funcs[Metric.query_size_B](data) + funcs[Metric.response_size_B](data) * SCALAR) / (1024)
                except Exception as e:
                    communication_KB = None

                # print(f"{name} {num_rows} ({runtime_s} s) ({communication_KB} KB)")
                with open(f"results/{name}.csv", "a") as f:
                    if TARGET_PAYLOAD_SIZE_B is None:
                        TARGET_PAYLOAD_SIZE_B = " "
                    if runtime_s is None:
                        runtime_s = " "
                    if communication_KB is None:
                        communication_KB = " "
                    f.write(f"{num_rows}, {TARGET_PAYLOAD_SIZE_B}, {runtime_s}, {communication_KB}\n")
            